In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
sns.set()
Candledf=pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
print(Candledf.tail())

                    ndatetime   open   high    low  close  volume
10300 2021-11-26 10:32:09.702  17366  17379  17326  17365   12000
10301 2021-11-26 10:54:27.533  17365  17381  17322  17350   12000
10302 2021-11-26 11:40:53.118  17350  17355  17310  17348   12000
10303 2021-11-26 12:39:01.486  17348  17354  17307  17329   12000
10304 2021-11-26 13:26:50.632  17329  17349  17318  17342    9564


In [2]:
(Candledf.close.shift(-20) / Candledf.close > 1).astype(int).tail()

10300    0
10301    0
10302    0
10303    0
10304    0
Name: close, dtype: int32

In [3]:
def triple_barrier(price, ub, lb, max_period):
    
    def end_price(s):
        return np.append(s[(s / s[0] > ub) | (s / s[0] < lb)], s[-1])[0]/s[0]
    
    r = np.array(range(max_period))
    
    def end_time(s):
        return np.append(r[(s / s[0] > ub) | (s / s[0] < lb)], max_period-1)[0]

    p = price.rolling(max_period).apply(end_price, raw=True).shift(-max_period+1)
    t = price.rolling(max_period).apply(end_time, raw=True).shift(-max_period+1)
    t = pd.Series([t.index[int(k+i)] if not math.isnan(k+i) else np.datetime64('NaT') 
                   for i, k in enumerate(t)], index=t.index).dropna()

    signal = pd.Series(0, p.index)
    signal.loc[p > ub] = 1
    signal.loc[p < lb] = -1
    ret = pd.DataFrame({'triple_barrier_profit':p, 'triple_barrier_sell_time':t, 'triple_barrier_signal':signal})

    return ret

In [8]:
ret = triple_barrier(Candledf.close, 1.07, 0.97, 87)


In [12]:
ret[ret.triple_barrier_signal < 0].count()
ret[ret.triple_barrier_profit>1]

,triple_barrier_profit,triple_barrier_sell_time,triple_barrier_signal
2,1.001240,88,0
4,1.004188,90,0
5,1.005333,91,0
6,1.002370,92,0
7,1.004267,93,0
...,...,...,...
10199,1.001018,10285,0
10200,1.000227,10286,0
10203,1.000226,10289,0
10206,1.001190,10292,0
